# Coupling BEM-BEM

<img src="1_Interfaz_EG.png" width="500" height="250">

To calculate the solvation energy of a solute using the BEM-BEM coupling, the following matrix system must be solved, which comes from solving the following PDE, including its border conditions:
\begin{equation} 
\left\{\begin{matrix} 
     -\epsilon_{m}\nabla^{2} \phi_{m}(x)=\sum_{i=1}^{n_{c}}Q_{i}\delta\left (x-x_{i} \right ) & x \ \in \ \Omega_{m} \\
     -\nabla^{2}\phi_{s}(x)+\kappa_{s}^{2}\phi_{s}(x)=0 & x \ \in \ \Omega_{s} \\
     \phi_{m}(x)= \phi_{s}(x) & x \ \in \ \Gamma \\
     \epsilon_{m}\partial_n \phi_{m}(x)= \epsilon_{s}\partial_n \phi_{s}(x) & x \ \in \ \Gamma 
\end{matrix}\right. 
\end{equation}

Where $\Omega_{m}$ is the solute domain, $\Omega_{s}$ is the solvent domain and $\Gamma$ is the interface. And its matrix system corresponds to:

\begin{equation}
\begin{bmatrix}
\frac{1}{2}I+K_{L} & -V_{L} \\
\frac{1}{2}I-K_{H} & \frac{\epsilon_{m}}{\epsilon_{s}}V_{H} \\
\end{bmatrix}\cdot
\begin{bmatrix}
\phi_{m}^{\Gamma} \\
\partial_n \phi_{m}^{\Gamma}\\
\end{bmatrix} =
\begin{bmatrix}
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma}-x_{i}\right \|} \\
0 \\
\end{bmatrix}
\end{equation}

And when solving the matrix system, the solvation energy is calculated as:
\begin{equation} 
    \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i} \left ( V_L\partial_n \phi_m^{\Gamma}(x_{i})-K_L\phi_m^{\Gamma}(x_{i}) \right )
\end{equation}   

## Implementation

First we import the main libraries such as Numpy, Trimesh and Bempp, as well as files in Python to read the solute files, which are in ".pqr" and ".off" format.

In [1]:
import bempp.api
import numpy as np 
import time
import trimesh
from readoff import *
from readpqr import *
start = time.time()

Next, the physical conditions of the problem in each region are defined, such as pemitivity and
Debye-Huckel parameter.

In [2]:
em = 4.    #[-] Interior electrical permittivity of the solute.
es = 80.   #[-] Exterior electrical permittivity of the solvent.
ks = 0.125  #[1/A] Inverse of the Debye-Huckel length of the fluid in the solvent.

Later, we choose the solution that we want to use to calculate the energy, where we import the surface mesh and the ".pqr" file to obtain the information on the radii, charges and position of the solute atoms.

In [3]:
Mesh1 ='Mallas_S/Sphere65R4.off'  #Path of the surface mesh of the solute for which the energy is to be calculated. In "off" format
PQR = 'PQR/Sphere5Q3.pqr'        #Path of the position and charges of the solute for which the energy is to be calculated. In "pqr" format
PC,Q,R = readpqr(PQR)  

In addition, we can choose the type of preconditioner to solve the matrix system, the type of assembly of the edge operators, parameters of the GMRES and the FMM if required.

In [4]:
#Choice to solve the matrix equation in BEM/BEM.
#True: Use the Mass Matrix preconditioner.
#False: Use the Block diagonal preconditioner.
SF = False  #Strong form. 
    
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =70      #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4   

Next, we generate the surface mesh of the solute.

In [5]:
vertices_0,faces_0 = read_off(Mesh1) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = vertices_0, faces= faces_0) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split)) #1 mesh means no cavity.

vertices_1 = mesh_split[0].vertices 
faces_1 = mesh_split[0].faces   
grid = bempp.api.grid.grid.Grid(vertices_1.transpose(), faces_1.transpose()) #Creation of the surface mesh.

Found 1 meshes


We make the Bempp function spaces of the potential and its derivative.

In [6]:
dirichl_space = bempp.api.function_space(grid, "P", 1)   #Electrostatic potential at the interface.
neumann_space = bempp.api.function_space(grid, "P", 1)   #Derived from the electrostatic potential at the interface.
print("DS dofs: {0}".format(dirichl_space.global_dof_count))
print("NS dofs: {0}".format(neumann_space.global_dof_count))  

DS dofs: 7908
NS dofs: 7908


We create the boundary operators that we need for each domain.

In [7]:
#Domain of the solute Ωm.
IL = bempp.api.operators.boundary.sparse.identity(dirichl_space, dirichl_space, dirichl_space) #1
KL = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, dirichl_space, dirichl_space, assembler=Assemble) #K
VL = bempp.api.operators.boundary.laplace.single_layer(neumann_space, dirichl_space, dirichl_space, assembler=Assemble) #V
#Domain of the solvent Ωs.
IH = bempp.api.operators.boundary.sparse.identity(dirichl_space, neumann_space, neumann_space) #1
if ks==0:
    KH = bempp.api.operators.boundary.laplace.double_layer(dirichl_space, neumann_space, neumann_space, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.laplace.single_layer(neumann_space, neumann_space, neumann_space, assembler=Assemble) #V
else:
    KH = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space, neumann_space, neumann_space, ks, assembler=Assemble) #K
    VH = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space, neumann_space, neumann_space, ks, assembler=Assemble) #V

Later, we create the Coulomb potential function $\phi_{c}^{\Gamma}$ and with that, we construct the vector right head side $b$ of formulation.
\begin{equation}
\phi_{c}^{\Gamma} =\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma}-x_{i}\right \|} \quad \quad
b=\begin{bmatrix}
\phi_{c}^{\Gamma} \\
0 \\
\end{bmatrix}
\end{equation}

In [8]:
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 /(4*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
U_c = bempp.api.GridFunction(dirichl_space, fun=U_c)

if SF==False:
    # Rhs in Ωm.
    rhs_M = (U_c).projections(dirichl_space)
    # Rhs in Ωs.
    rhs_S = np.zeros(neumann_space.global_dof_count)
    # The combination of Rhs.
    rhs = np.concatenate([rhs_M, rhs_S])
else:    
    rhs = [I1*U_c, 0*I2*U_c]

And we construct the global left 2x2 matrix $A$ of the formulation.   
\begin{equation}
A=\begin{bmatrix}
\frac{1}{2}I+K_{L} & -V_{L} \\
\frac{1}{2}I-K_{H} & \frac{\epsilon_{m}}{\epsilon_{s}}V_{H} \\
\end{bmatrix}
\end{equation}

In [9]:
if SF==False:
    #Position of the 2x2 matrix.
    blocks = [[None,None],[None,None]] 
    blocks[0][0] = (0.5*IL + KL).weak_form() 
    blocks[0][1] = -VL.weak_form()            
    blocks[1][0] = (0.5*IH - KH).weak_form()  
    blocks[1][1] = (em/es)*VH.weak_form()        
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))   
    #Block diagonal preconditioner for BEM.
    from preconditioners import *
    P = BlockDiagonal_2x2(dirichl_space, neumann_space, blocks, es, em, ks) 
else:   
    #Position of the 2x2 matrix.
    blocks = bempp.api.BlockedOperator(2,2)    
    blocks[0,0] = (0.5*IL + KL)  
    blocks[0,1] = -VL            
    blocks[1,0] = (0.5*IH - KH) 
    blocks[1,1] = (em/es)*VH     

Later, we solve the matrix system $Ax=b$ with GMRES together with its respective preconditioning an efficient solution. The solution is then divided into the parts associated with $\phi_{m}^{\Gamma}$ and $\partial_{n}\phi_{m}^{\Gamma}$. 

In [10]:
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)

# Solution by GMRES.
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
    end1 = time.time() 
else:
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True, tol=Tol, restart=Res)
    end1 = time.time()
    
# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))

Number of GMRES iterations: 28
Total time in GMRES: 15.75 [s]


Next, we make Bempp functions from the solution of $\phi_{m}^{\Gamma}$ and $\partial_{n}\phi_{m}^{\Gamma}$.

In [11]:
if SF==False:
    # Solution for Dirichlet data.
    soln_u = soln[:dirichl_space.global_dof_count]
    dirichlet_fun = bempp.api.GridFunction(dirichl_space, coefficients=soln_u)    
    # Solution for Neumann data.
    soln_du = soln[dirichl_space.global_dof_count:] 
    neumann_fun = bempp.api.GridFunction(neumann_space, coefficients=soln_du)
else:
    dirichlet_fun = soln[0]
    neumann_fun = soln[1]

Finally we calculate the solvation energy for the BEM-BEM coupling, using the equation of the potential at the position of the solute atoms.
\begin{equation}
\phi_m(x) =V_L\partial_{n}\phi_m^{\Gamma}(x)-K_L\phi_m^{\Gamma}(x)  \quad \quad
 \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i}\phi_{m}(x_i) 
\end{equation}   

In [12]:
#Result of the total solvation energy.
VF = bempp.api.operators.potential.laplace.single_layer(neumann_space, np.transpose(PC)) 
KF = bempp.api.operators.potential.laplace.double_layer(dirichl_space, np.transpose(PC))
uF = VF*neumann_fun - KF*dirichlet_fun
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -55.901478 [kCal/mol]
Total time: 189.62 [s]
